In [1]:

import os
import sys
import numpy as np
import pandas as pd
from scipy.io import mmread
from scipy.linalg import hessenberg
import scipy.linalg as sl
sys.path.append("../qr")

from qr import *
import sympy as sp

In [8]:
a = complex_matrix(5, 1, 2)
b = np.array([0, 0])
b.size

2

In [10]:
pd.DataFrame(a)

,0,1,2,3,4
0,1.896277+1.893681j,1.511920+1.974714j,1.363502+1.046901j,1.894698+1.312846j,1.861828+1.608494j
1,1.579803+1.294133j,1.390823+1.065456j,1.353642+1.967684j,1.068047+1.344476j,1.926517+1.190223j
2,1.781252+1.381885j,1.102028+1.226897j,1.342858+1.383427j,1.552791+1.621857j,1.014538+1.761267j
3,1.772020+1.063565j,1.084608+1.996098j,1.301553+1.734940j,1.123615+1.998403j,1.803043+1.643719j
4,1.162048+1.894058j,1.631407+1.978184j,1.072105+1.185666j,1.374046+1.296028j,1.437880+1.484406j


In [14]:
sqrt(2)

NameError: name 'sqrt' is not defined

In [29]:
%%timeit
a ** 2

839 ns ± 4.31 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [30]:
%%timeit
np.power(a, 2)

1.19 µs ± 4.25 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [192]:
a = np.random.default_rng().integers(0, 10, (4, 5))
b = np.random.default_rng().integers(0, 10, (3, 3))
a @ b 

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 5)

In [189]:
path = "../test_matrices"
mat_1_file = "west0381"
ext = ".mtx.gz"

mat = mmread(os.path.join(path, "".join((mat_1_file, ext))))

In [190]:
m = mat.toarray()
m

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [138]:
import mpmath as mpm

mpm.dps = 15

def complex_matrix(n: int, a: float, b: float) -> np.ndarray:
	if a >= b:
		raise ValueError("Required: b > a")
	
	r = (b - a) * np.random.default_rng().random(size = (n, n)) + a
	c = (b - a) * np.random.default_rng().random(size = (n, n)) + a
	m = r + 1j * c
	
	return m.astype(np.complex128)

def householder_reflector(x: np.array):
	"""
	Produces the Householder
	vector based on the input 
	vector x. The householder 
 	vector acts as:
 
	|a_1|		|alpha|	
	|a_2|	->	|0|
	|a_3|		|0|

	Parameters
	----------
	x:	
		A numpy array who's entries
		after the 1st element needs to 
		be 0ed. 
  
	Returns
	-------
	A numpy array that acts as the 
	Householder vector. 
	"""
	u = x.copy()
	
	rho = -np.exp(1j * np.angle(u[0]), dtype = np.complex128)

	# Set the Householder vector
	# to u = u \pm alpha e_1 to 
	# avoid cancellation.
	u[0] -= rho * mpm.norm(u)
 
	# Vector needs to have 1 
	# in the 2nd dimension.
	# print(u)
	return u.reshape(-1, 1)

def hessenberg_transform_1(M: np.ndarray) -> np.ndarray:
	"""
	Converts a given matrix to 
	Hessenberg form using
	Houeholder transformations.

	Parameters
	----------
	M:	
 		A complex square 
		numpy 2darray.

	Returns
	-------
	A tuple consisting of numpy
 	2-D arrays which are the 
	hessenberg form and the 
	permutation matrix.
	"""
	h = M.copy()
	n = np.array(h.tolist()).shape[0]
	u = np.eye(n, dtype = np.complex128)
	householder_vectors = list()
 
	# MAIN LOOP.
	for l in range(n - 2):
		# Get the Householder vector for h.
		t = householder_reflector(h[l + 1 :, l])

		# Norm**2 of the Householder vector.
		t_norm_squared = t.conj().T @ t
  
		# p = np.eye(h[l + 1:, l].shape[0]) - 2 * (np.outer(t, t)) / t_norm_squared

		# # Resize and refactor the Householder matrix.
		# p = np.pad(p, ((l + 1, 0), (l + 1, 0)), mode = "constant", constant_values = ((0, 0), (0, 0)))
		# for k in range(l + 1):
		# 	p[k, k] = 1

		# Perform a similarity transformation on h
		# using the Householder matrix.
		# h = p @ h @ p.
  
		# --- REAL --- #
		# Left multiplication by I - 2uu^{*}.
		# h_real[l + 1 :, l :] -= 2 * (t @ (t.conj().T @ h_real[l + 1 :, l :])) / t_norm_squared
		# Right multiplication by I - 2uu^{*}.
		# h_real[ :, l + 1 :] -= 2 * ((h[ :, l + 1 :] @ t) @ t.conj().T) / t_norm_squared
		# print(f"{np.array(h[l + 1 :, l :].tolist()).shape = }")
		# print(f"{np.array(t.transpose_conj().tolist()).shape = }")
		# print(f"{np.array((t.transpose_conj() * h[l + 1 :, l :]).tolist()).shape = }")
		factor = 2 / t_norm_squared
  
		h[l + 1 :, l :] -= factor * (t @ (t.conj().T @ h[l + 1 :, l :]))

		# --- IMAGINARY --- #
		# Left multiplication by I - 2uu^{*}.
		# h_imag[l + 1 :, l :] -= 2 * (t @ (t.conj().T @ h_imag[l + 1 :, l :])) / t_norm_squared
		# Right multiplication by I - 2uu^{*}.
		# h_imag[ :, l + 1 :] -= 2 * ((h[ :, l + 1 :] @ t) @ t.conj().T) / t_norm_squared
		h[ :, l + 1 :] -= factor * ((h[ :, l + 1 :] @ t) @ t.conj().T)
		
		# Force elements below main
		# subdiagonal to be 0.
		h[l + 2 :, l] = 0.0

		# Store the transformations 
		# to compute u.
		householder_vectors.append(t)
			
	# Store the transformations.
	for k in reversed(range(n - 2)):
		t = householder_vectors[k]
		t_norm_squared = np.dot(t.conj().T, t)
		u[k + 1 :, k + 1 :] = 2 * t * (t.conj().T @ u[k + 1 :, k + 1 :]) / t_norm_squared

	# h = h_real + 1j * h_imag
	return h, u

In [186]:
n = 1000
a = 10.0
b = 20.0
# m = complex_matrix(n, a, b)
# M = mpm.matrix(m.tolist())
hess_from_alg, _ = hessenberg_transform_1(m)
hess_from_scipy = hessenberg(m) 

../qr/hessenberg.py:51: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [187]:
%%capture cap --no-stderr

pd.options.display.max_columns = 200
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 1000)

# print(f" Hessenberged:\n {pd.DataFrame(hess_alg)}")
# print(f"Hessenberged (scipy):\n {pd.DataFrame(hess_from_scipy)}")
eigs = np.sort(np.linalg.eig(hess_from_alg)[0])
eigs_scipy = np.sort(np.linalg.eig(hess_from_scipy)[0])
print(f"Eigs:\n {pd.DataFrame(np.vstack([eigs, eigs_scipy]).T)}")
print(f"Equality of eigs: {np.testing.assert_allclose(eigs_scipy, eigs, rtol = 1e-6)}")

In [188]:
with open("test_ipynb_output.txt", "w") as f:
    f.write(cap.stdout)

In [40]:
sl.blas.daxpy([1, 2, 3], [1, 2, 3], a = 0.5)

array([1.5, 3. , 4.5])

In [45]:
def sign(z: complex) -> complex:
		if z == 0:
			return 1
		return z / abs(z)

sign(-2.0 + 2.j)

(-0.7071067811865475+0.7071067811865475j)

In [98]:
a = np.array([1.00345345, 2, 1, -1, 2])
b = np.array([1.00354, 2, 1, -1, 2])
[i for i, _ in enumerate(a) if np.isclose(_, 1.00354, 1e-3)]

0

In [84]:
np.round(1.011234, 1) == 1.0

True

In [95]:
dec = 6
a = f"{0:.{dec}f}"
print(float(a))

SyntaxError: invalid decimal literal (3669851116.py, line 2)

In [99]:
np.prod([1, 2, 3, 4])

24